# 1. environment setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q --upgrade pip
!pip install -q \
  transformers>=4.41.0 \
  accelerate datasets peft bitsandbytes trl sentence-transformers \
  fsspec==2025.3.2

# 2. model training

주요 구성요소

- tokenizer에 apply_chat_template() 사용
- tokenize_and_mask() 함수로 손실 계산 시 user 발화는 무시
- SFTTrainer로 학습

In [ ]:
from datasets import Dataset
import json
from transformers import AutoTokenizer

# tokenizer 정의
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# padding 토큰 설정 (이거 없으면 오류 ㅈㄴ)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2. json 파일 로드
with open("/content/drive/MyDrive/Colab/Finetuning_medical3/dialogues_medical3.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# 3. Hugging Face Dataset 변환
dataset = Dataset.from_list(raw_data)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

채팅 템플릿 적용

In [ ]:
def format_chat_template(row, tokenizer): # 각 input/output 샘플을 LLaMA-3의 chat format으로 변환하는 역할
    messages = [
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        "사용자는 자신의 이름, 전화번호, 진료 예약 정보를 제공하고, 갑작스러운 일정으로 진료 시간을 변경하고자 전화한 상황입니다."
        "사용자의 진료 시간을 변경해주려면 사용자의 이름과 전화번호를 받아야 합니다.."
        "예약 정보를 확인하고, 변경 가능한 시간대를 안내한 뒤 사용자가 선택한 시간으로 예약을 변경해 주세요. "
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},
        {"role": "user", "content": row["input"]},    # 사용자 발화
        {"role": "assistant", "content": row["output"]}   # 모델이 학습할 응답
    ]
    #     # tokenizer로 LLaMA 채팅 포맷 변환 (실제 텍스트로 렌더링)
    formatted_chat = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False  # assistant 응답까지 포함 (학습용)
    )
    return {"text": formatted_chat}

formatted_dataset = dataset.map(lambda row: format_chat_template(row, tokenizer))


# 확인용 코드
print("- - - - - - - - 데이터 확인 - - - - - - - - ")
print(formatted_dataset[0]['text'])

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

- - - - - - - - 데이터 확인 - - - - - - - - 
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 경희내과의 병원 콜센터 상담원입니다.사용자는 자신의 이름, 전화번호, 진료 예약 정보를 제공하고, 갑작스러운 일정으로 진료 시간을 변경하고자 전화한 상황입니다.사용자의 진료 시간을 변경해주려면 사용자의 이름과 전화번호를 받아야 합니다..예약 정보를 확인하고, 변경 가능한 시간대를 안내한 뒤 사용자가 선택한 시간으로 예약을 변경해 주세요. 모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

안녕하세요. 오늘 경희내과에 예약이 되어 있는데요, 시간이 좀 어려워서 변경 문의드리려고요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

네, 안녕하세요. 확인 도와드릴게요. 성함이 어떻게 되실까요?<|eot_id|>


마스킹 포함 토크나이징

In [ ]:
def tokenize_and_mask(example):
    # tokenizer를 이용해 텍스트를 토크나이즈하고 텐서 반환
    encoding = tokenizer(
        example["text"],
        max_length=1024,
        padding="max_length",
        truncation=True,             # 길면 자르기
        return_tensors="pt"          # PyTorch 텐서 형태로 반환
    )

    # input_ids는 토큰 번호 리스트
    input_ids = encoding["input_ids"][0].tolist()
    # labels는 input_ids를 복사해서 사용
    labels = input_ids.copy()
    # assistant 응답이 시작되는 토큰 ID를 찾아냄.
    assistant_token_id = tokenizer.convert_tokens_to_ids("<|assistant|>")
    try:
        start = input_ids.index(assistant_token_id)
    except ValueError:
        # 토큰이 없을 경우 fallback (전체 학습으로 처리)
        start = 0

    # assistant 이전까지는 loss 계산에서 제외 (마스킹 처리)
    # 모델은 user 입력을 읽기만 하고, assistant 응답부터 예측하도록 학습해야 함. 그래서 assistant 이전은 제외
    for i in range(start):
        labels[i] = -100

    # 최종적으로 input_ids, attention_mask, labels 반환
    return {
        "input_ids": input_ids,
        "attention_mask": encoding["attention_mask"][0].tolist(),
        "labels": labels,
    }

# 전체 formatted dataset에 대해 토크나이징 및 마스킹 적용
tokenized_dataset = formatted_dataset.map(tokenize_and_mask)


Map:   0%|          | 0/373 [00:00<?, ? examples/s]

모델 및 LoRA 설정 -> *학습*

LoRA(Low-Rank Adaptation) : 딥러닝 모델, 특히 GPT나 LLaMA같은 LLM을 효율적으로 미세조정하기 위한 경량화 기법

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer
import torch
torch.cuda.empty_cache()

# 모델 로딩
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# LoRA 설정 (적은 파라미터로 미세조정)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,     # 10
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,    # 4
    learning_rate=2e-4,
    save_strategy="steps",      # epoch
    save_steps=500,
    save_total_limit=1,     # check point로 저장되는거를 최근 1개 빼고는 다 삭제함.
    logging_steps=50,
    fp16=True,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03
)

# SFTTrainer로 학습 객체 구성
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
    max_seq_length=512,   # 1024
    packing=False   # 여러 샘플을 한 시퀀스로 이어붙인다캄
)

# 학습 시작
trainer.train()

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wjd_bin217 (wjd_bin217-kyung-hee-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,1.808800
100,0.473500
150,0.406600
200,0.347400
250,0.268400
300,0.311100
350,0.271500
400,0.249800
450,0.233200
500,0.242000


TrainOutput(global_step=1865, training_loss=0.23247644862924122, metrics={'train_runtime': 973.3842, 'train_samples_per_second': 1.916, 'train_steps_per_second': 1.916, 'total_flos': 8.607372826116096e+16, 'train_loss': 0.23247644862924122, 'epoch': 5.0})

# 3. model save

In [ ]:
# LoRA adapter만 따로 저장
from peft import PeftModel

# adapter만 저장
model.save_pretrained("DialogueGenModel_lora_adapter")
tokenizer.save_pretrained("DialogueGenModel_lora_adapter")

import os
print(os.listdir("DialogueGenModel_lora_adapter"))

['model-00002-of-00002.safetensors', 'tokenizer_config.json', 'tokenizer.json', 'chat_template.jinja', 'model-00001-of-00002.safetensors', 'generation_config.json', 'config.json', 'model.safetensors.index.json', 'special_tokens_map.json']


# 4. uploading to Hugging Face


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
print(type(model))  # <class 'peft.model.PeftModelForCausalLM'>이면 OK, 아니라면 명시적으로 LoRㅁ 구성


<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# LoRA 적용 전의 원본 모델 ID
base_model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

# base model과 tokenizer 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# PeftModel은 base model 위에 LoRA adapter만 얹는 구조

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel
from huggingface_hub import create_repo, upload_folder
from peft import get_peft_model

# Hugging Face Repo 정보
repo_id = "wjdbin217/DialogueGenModel_finetuning_medical3"

# LoRA adapter 저장 경로
adapter_path = "DialogueGenModel_lora_adapter"


# LoRA adapter 저장 (trainer.model이 PerftModel인 상태여야 adapter_config.json이 저장됨.)
trainer.model.save_pretrained("DialogueGenModel_lora_adapter")
tokenizer.save_pretrained("DialogueGenModel_lora_adapter")


# 나중에 adapter_config.json 파일이 있는지 꼭 확인

# 업로드
upload_folder(
    repo_id=repo_id,
    folder_path=adapter_path,
    path_in_repo=".",
    commit_message="Upload LoRA adapter for DialogueGenModel fine-tuning. standart model : llama-3-Korean-Bllossom-8B"
)


Uploading...:   0%|          | 0.00/7.88G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wjdbin217/DialogueGenModel_finetuning_medical3/commit/8a6c0dc6388ac9010fc216baf9928f23620ff3b8', commit_message='Upload LoRA adapter for DialogueGenModel fine-tuning. standart model : llama-3-Korean-Bllossom-8B', commit_description='', oid='8a6c0dc6388ac9010fc216baf9928f23620ff3b8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wjdbin217/DialogueGenModel_finetuning_medical3', endpoint='https://huggingface.co', repo_type='model', repo_id='wjdbin217/DialogueGenModel_finetuning_medical3'), pr_revision=None, pr_num=None)

In [ ]:
# 업로드 여부 확인
from huggingface_hub import list_repo_files

repo_id = "wjdbin217/DialogueGenModel_finetuning_medical3"
files = list_repo_files(repo_id)

print("업로드된 파일 목록 :")
for f in files:
    print(f)


업로드된 파일 목록 :
.gitattributes
README.md
adapter_config.json
adapter_model.safetensors
chat_template.jinja
config.json
generation_config.json
model-00001-of-00002.safetensors
model-00002-of-00002.safetensors
model.safetensors.index.json
special_tokens_map.json
tokenizer.json
tokenizer_config.json


# 5. loading from Hugging Face

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# 1. Hugging Face에 올린 LoRA adapter 경로
adapter_repo = "wjdbin217/DialogueGenModel_finetuning_medical3"

# 2. LoRA 원본 base model 불러오기
base_model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

tokenizer = AutoTokenizer.from_pretrained(adapter_repo)
# 일반적으로 tokenizer은 base model 기준으로 가져오긴 함. 지금은 adapter repo에서 가져옴.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# 3. LoRA adapter 적용
model = PeftModel.from_pretrained(base_model, adapter_repo)


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/869 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [ ]:
# 4. Inference
messages = [
        {"role": "system", "content": "당신은 경희내과의 병원 콜센터 상담원입니다."
        "사용자는 자신의 이름, 전화번호, 진료 예약 정보를 제공하고, 갑작스러운 일정으로 진료 시간을 변경하고자 전화한 상황입니다."
        "사용자의 진료 시간을 변경해주려면 사용자의 이름과 전화번호를 받아야 합니다.."
        "예약 정보를 확인하고, 변경 가능한 시간대를 안내한 뒤 사용자가 선택한 시간으로 예약을 변경해 주세요. "
        "모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다."},
    {"role": "user", "content": "경희내과 맞나요? 저 오늘 예약한 홍길동인데, 시간을 좀 바꿔야 할 것 같아요."},
    {"role": "assistant", "content": "네, 고객님. 경희내과입니다. 본인 확인되셨고요. 어떤 시간대로 변경을 희망하시나요?"},
    {"role": "user", "content": "네. 오후로요. 딱 정해진 시간은 없는데... 혹시 추천해주실 만한 시간 있으세요?"},
]

# Chat 템플릿 적용
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,     # 뒤에 잘리면 조정 필요
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

# 출력
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("- - - - - - - - - - - - - - - - - 전체 응답 - - - - - - - - - - - - - - - - - ")
print(result)
# print("- - - - - - - - - - - - - - - - - 잘라낸 부분만 - - - - - - - - - - - - - - - - - ")
# print(result.split("<|assistant|>")[-1].strip())


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


- - - - - - - - - - - - - - - - - 전체 응답 - - - - - - - - - - - - - - - - - 
system

당신은 경희내과의 병원 콜센터 상담원입니다.사용자는 자신의 이름, 전화번호, 진료 예약 정보를 제공하고, 갑작스러운 일정으로 진료 시간을 변경하고자 전화한 상황입니다.사용자의 진료 시간을 변경해주려면 사용자의 이름과 전화번호를 받아야 합니다..예약 정보를 확인하고, 변경 가능한 시간대를 안내한 뒤 사용자가 선택한 시간으로 예약을 변경해 주세요. 모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다.user

경희내과 맞나요? 저 오늘 예약한 홍길동인데, 시간을 좀 바꿔야 할 것 같아요.assistant

네, 고객님. 경희내과입니다. 본인 확인되셨고요. 어떤 시간대로 변경을 희망하시나요?user

네. 오후로요. 딱 정해진 시간은 없는데... 혹시 추천해주실 만한 시간 있으세요?assistant

네, 성함과 연락처 확인되셨고요. 지금 오후 2시 30분, 3시 50분, 5시 10분 진료가 가능합니다.
- - - - - - - - - - - - - - - - - 잘라낸 부분만 - - - - - - - - - - - - - - - - - 
system

당신은 경희내과의 병원 콜센터 상담원입니다.사용자는 자신의 이름, 전화번호, 진료 예약 정보를 제공하고, 갑작스러운 일정으로 진료 시간을 변경하고자 전화한 상황입니다.사용자의 진료 시간을 변경해주려면 사용자의 이름과 전화번호를 받아야 합니다..예약 정보를 확인하고, 변경 가능한 시간대를 안내한 뒤 사용자가 선택한 시간으로 예약을 변경해 주세요. 모든 응답은 정중하고 친절한 말투로 하며, 병원 콜센터 상담원이 실제 통화에서 말하듯 자연스럽고 공손하게 작성해야 합니다.user

경희내과 맞나요? 저 오늘 예약한 홍길동인데, 시간을 좀 바꿔야 할 것 같아요.assistant

네, 고객님. 경희내과입니다. 

## training data   CSV -> JSON 변환
data를 json 형식으로 변환하기 위해 사용한 일회성 코드입니다.

In [ ]:
import csv
import json

# 파일 경로 설정 (Colab 기준)
csv_path = "/content/drive/MyDrive/Colab/Finetuning_medical3/dialogues_medical3.csv"
json_path = "/content/drive/MyDrive/Colab/Finetuning_medical3/dialogues_medical3.json"

# 변환 리스트 초기화
converted_data = []

# CSV 읽기 (쉼표, 줄바꿈 자동 처리)
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        prompt = row.get("prompt", "").strip()
        completion = row.get("completion", "").strip()

        # 빈 셀 무시
        if prompt and completion:
            converted_data.append({
                "input": prompt,
                "output": completion
            })

# JSON 저장
with open(json_path, "w", encoding="utf-8") as jsonfile:
    json.dump(converted_data, jsonfile, ensure_ascii=False, indent=2)

print(f"변환 완료! JSON 저장 경로 : {json_path}")
print(f"총 샘플 수: {len(converted_data)}개")


변환 완료! JSON 저장 경로 : /content/drive/MyDrive/Colab/Finetuning_medical3/dialogues_medical3.json
총 샘플 수: 373개


In [ ]:
import json

# JSON 경로
json_path = "/content/drive/MyDrive/Colab/Finetuning_medical3/dialogues_medical3.json"

# 파일 불러오기
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 몇 개만 출력 (예: 앞에서 5개)
print(" - - - - - - - - - - 샘플 예시 - - - - - - - - - - ")
for i, sample in enumerate(data[:5], 1):
    print(f"\n 샘플 {i}")
    print(f"[User Input]   {sample['input']}")
    print(f"[Assistant Output]   {sample['output']}")


 - - - - - - - - - - 샘플 예시 - - - - - - - - - - 

 샘플 1
[User Input]   안녕하세요. 오늘 경희내과에 예약이 되어 있는데요, 시간이 좀 어려워서 변경 문의드리려고요.
[Assistant Output]   네, 안녕하세요. 확인 도와드릴게요. 성함이 어떻게 되실까요?

 샘플 2
[User Input]   홍길동입니다. 오늘 오전에 예약돼 있었어요.
[Assistant Output]   잠시만요.확인되었습니다. 오늘 오전 11시, 내과 진료로 예약되어 있으시네요. 변경 원하시는 시간대 있으신가요?

 샘플 3
[User Input]   오후 시간대로 바꾸고 싶은데요, 가능한 시간 알려주실 수 있을까요?
[Assistant Output]   네, 오후에는 2시 30분, 4시 30분 비어 있습니다. 어떤 시간으로 변경해드릴까요?

 샘플 4
[User Input]   2시 30분으로 바꿔주세요.
[Assistant Output]   네 변경해드렸습니다. 다른 문의사항 있으실까요?

 샘플 5
[User Input]   아니요, 없어요.
[Assistant Output]   네, 감사합니다.
